# 01 — Data Profiling & Statistical Analysis (BTCUSDT)

**Goal:** Understand the statistical properties of our market data before building signals.

**Questions:**
1. Return distributions at 1m/5m — tail behavior, skewness, kurtosis
2. Autocorrelation of returns — is there serial dependence?
3. Volatility clustering — how persistent are vol regimes?
4. Intraday seasonality — volume/volatility by hour of day
5. Cross-exchange correlation — price relationships & lead-lag across Binance, Bybit, OKX
6. Volume imbalance — predictive of forward returns?
7. Binance metrics — OI, funding, L/S ratios

**Data:** Daily-partitioned parquet files from `build_parquet.py`  
**Period:** 2025-11-01 → 2026-01-31 (92 days)  
**Sources:** Bybit, Binance, OKX (futures + spot each)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pathlib import Path
from scipy import stats

plt.rcParams['figure.figsize'] = (14, 5)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

PARQUET_DIR = Path('../parquet')
SYMBOL = 'BTCUSDT'

SOURCES = ['bybit_futures', 'bybit_spot', 'binance_futures', 'binance_spot',
           'okx_futures', 'okx_spot']
FUTURES_SOURCES = ['bybit_futures', 'binance_futures', 'okx_futures']
SPOT_SOURCES = ['bybit_spot', 'binance_spot', 'okx_spot']

SOURCE_LABELS = {
    'bybit_futures': 'Bybit Futures',
    'bybit_spot': 'Bybit Spot',
    'binance_futures': 'Binance Futures',
    'binance_spot': 'Binance Spot',
    'okx_futures': 'OKX Futures',
    'okx_spot': 'OKX Spot',
}
SOURCE_COLORS = {
    'bybit_futures': '#FF6B00',
    'bybit_spot': '#FFB366',
    'binance_futures': '#F0B90B',
    'binance_spot': '#F0D96B',
    'okx_futures': '#00C4B4',
    'okx_spot': '#66E0D6',
}

# ---------------------------------------------------------------------------
# Loaders for daily-partitioned parquet
# ---------------------------------------------------------------------------

def load_ohlcv(symbol, interval, source):
    """Load all daily OHLCV parquet files for a source, concatenated."""
    ohlcv_dir = PARQUET_DIR / symbol / 'ohlcv' / interval / source
    if not ohlcv_dir.exists():
        return pd.DataFrame()
    files = sorted(ohlcv_dir.glob('*.parquet'))
    if not files:
        return pd.DataFrame()
    df = pd.concat([pd.read_parquet(f) for f in files], ignore_index=True)
    df = df.sort_values('timestamp_us').reset_index(drop=True)
    df['datetime'] = pd.to_datetime(df['timestamp_us'], unit='us', utc=True)
    df['returns'] = df['close'].pct_change()
    df['log_returns'] = np.log(df['close'] / df['close'].shift(1))
    return df

def load_ohlcv_daterange(symbol, interval, source, start_date, end_date):
    """Load OHLCV for a specific date range (inclusive)."""
    ohlcv_dir = PARQUET_DIR / symbol / 'ohlcv' / interval / source
    if not ohlcv_dir.exists():
        return pd.DataFrame()
    dates = pd.date_range(start_date, end_date)
    files = [ohlcv_dir / f'{d.strftime("%Y-%m-%d")}.parquet' for d in dates]
    files = [f for f in files if f.exists()]
    if not files:
        return pd.DataFrame()
    df = pd.concat([pd.read_parquet(f) for f in files], ignore_index=True)
    df = df.sort_values('timestamp_us').reset_index(drop=True)
    df['datetime'] = pd.to_datetime(df['timestamp_us'], unit='us', utc=True)
    df['returns'] = df['close'].pct_change()
    df['log_returns'] = np.log(df['close'] / df['close'].shift(1))
    return df

def load_metrics(symbol):
    """Load all daily Binance futures metrics."""
    metrics_dir = PARQUET_DIR / symbol / 'binance' / 'metrics'
    if not metrics_dir.exists():
        return pd.DataFrame()
    files = sorted(metrics_dir.glob('*.parquet'))
    if not files:
        return pd.DataFrame()
    df = pd.concat([pd.read_parquet(f) for f in files], ignore_index=True)
    df = df.sort_values('timestamp_us').reset_index(drop=True)
    df['datetime'] = pd.to_datetime(df['timestamp_us'], unit='us', utc=True)
    return df

def count_trades(symbol, source):
    """Count total trades and get date range without loading all data."""
    trades_dir = PARQUET_DIR / symbol / 'trades' / source
    if not trades_dir.exists():
        return 0, None, None, None, None
    files = sorted(trades_dir.glob('*.parquet'))
    if not files:
        return 0, None, None, None, None
    total = 0
    price_min, price_max = float('inf'), float('-inf')
    for f in files:
        df = pd.read_parquet(f, columns=['price'])
        total += len(df)
        price_min = min(price_min, df['price'].min())
        price_max = max(price_max, df['price'].max())
    first_date = files[0].stem
    last_date = files[-1].stem
    return total, first_date, last_date, price_min, price_max

print(f'Ready. Symbol: {SYMBOL}')
print(f'Parquet dir: {PARQUET_DIR / SYMBOL}')

## 1. Data Overview

Quick summary of what we have: row counts, time ranges, price ranges per source.

In [ ]:
print(f'{"=" * 80}')
print(f'  {SYMBOL} — Data Overview')
print(f'{"=" * 80}')

# Trades summary
print(f'\n  Raw Trades:')
print(f'  {"Source":25s} {"Trades":>14s} {"Date Range":>25s} {"Price Range":>25s}')
print(f'  {"─" * 92}')
for src in SOURCES:
    total, d0, d1, pmin, pmax = count_trades(SYMBOL, src)
    if total == 0:
        print(f'  {SOURCE_LABELS[src]:25s}  (no data)')
        continue
    print(f'  {SOURCE_LABELS[src]:25s} {total:>14,} {d0} → {d1:>10s} '
          f'${pmin:>10,.2f} – ${pmax:,.2f}')

# OHLCV summary
for interval in ['1m', '1h']:
    print(f'\n  OHLCV {interval}:')
    print(f'  {"Source":25s} {"Bars":>10s} {"Total Volume":>16s} {"Total Trades":>14s}')
    print(f'  {"─" * 70}')
    for src in SOURCES:
        df = load_ohlcv(SYMBOL, interval, src)
        if df.empty:
            print(f'  {SOURCE_LABELS[src]:25s}  (no data)')
            continue
        print(f'  {SOURCE_LABELS[src]:25s} {len(df):>10,} {df["volume"].sum():>16,.0f} '
              f'{df["trade_count"].sum():>14,}')

# Metrics summary
metrics = load_metrics(SYMBOL)
if not metrics.empty:
    print(f'\n  Binance Metrics: {len(metrics):,} rows  '
          f'{metrics["datetime"].min().strftime("%Y-%m-%d")} → '
          f'{metrics["datetime"].max().strftime("%Y-%m-%d")}')

## 2. Return Distributions

Analyze return distributions at 1m and 5m to understand:
- **Kurtosis** (fat tails = more extreme moves than normal)
- **Skewness** (asymmetry = directional bias)
- How far from Gaussian the returns are

In [ ]:
print(f'{SYMBOL} — Return Statistics')
print(f'{"─" * 100}')
print(f'{"Source":25s} {"Interval":>8s} {"Mean":>12s} {"Std":>12s} '
      f'{"Skew":>8s} {"Kurt":>8s} {"Min":>10s} {"Max":>10s} {"JB p-val":>10s}')
print(f'{"─" * 100}')

for interval in ['1m', '5m']:
    for src in SOURCES:
        df = load_ohlcv(SYMBOL, interval, src)
        if df.empty:
            continue
        r = df['returns'].dropna()
        jb_stat, jb_p = stats.jarque_bera(r)
        print(f'{SOURCE_LABELS[src]:25s} {interval:>8s} '
              f'{r.mean():>12.6f} {r.std():>12.6f} '
              f'{r.skew():>8.3f} {r.kurtosis():>8.2f} '
              f'{r.min():>10.5f} {r.max():>10.5f} '
              f'{jb_p:>10.2e}')
    print()

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle(f'{SYMBOL} — Return Distributions (1m top, 5m bottom)', fontsize=14, fontweight='bold')

for i, interval in enumerate(['1m', '5m']):
    for j, src in enumerate(FUTURES_SOURCES):
        ax = axes[i, j]
        df = load_ohlcv(SYMBOL, interval, src)
        if df.empty:
            continue
        r = df['returns'].dropna()
        
        ax.hist(r, bins=200, density=True, alpha=0.7,
                color=SOURCE_COLORS[src], label=SOURCE_LABELS[src])
        
        # Normal overlay
        x = np.linspace(r.quantile(0.001), r.quantile(0.999), 300)
        ax.plot(x, stats.norm.pdf(x, r.mean(), r.std()),
                'r--', alpha=0.8, label='Normal')
        
        ax.set_title(f'{SOURCE_LABELS[src]} — {interval} returns')
        ax.set_xlabel('Return')
        ax.legend(fontsize=8)
        ax.set_xlim(-0.01, 0.01)

plt.tight_layout()
plt.show()

## 3. Autocorrelation of Returns

If returns have significant autocorrelation at short lags, there's direct evidence of predictability.
- Positive ACF → momentum
- Negative ACF → mean reversion

In [ ]:
from statsmodels.tsa.stattools import acf

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle(f'{SYMBOL} — Autocorrelation of Returns', fontsize=14, fontweight='bold')

for i, interval in enumerate(['1m', '5m']):
    for j, src in enumerate(FUTURES_SOURCES):
        ax = axes[i, j]
        df = load_ohlcv(SYMBOL, interval, src)
        if df.empty:
            continue
        r = df['returns'].dropna().values
        
        nlags = min(60, len(r) // 4)
        acf_vals, confint = acf(r, nlags=nlags, alpha=0.05)
        
        ax.bar(range(nlags + 1), acf_vals, width=0.6,
               color=SOURCE_COLORS[src], alpha=0.7)
        ax.fill_between(range(nlags + 1),
                       confint[:, 0] - acf_vals,
                       confint[:, 1] - acf_vals,
                       alpha=0.15, color='blue')
        ax.axhline(0, color='black', linewidth=0.5)
        ax.set_title(f'{SOURCE_LABELS[src]} — {interval} returns ACF')
        ax.set_xlabel('Lag')
        ax.set_ylabel('ACF')

plt.tight_layout()
plt.show()

# ACF of absolute returns (volatility clustering)
fig, axes = plt.subplots(1, 3, figsize=(18, 4))
fig.suptitle(f'{SYMBOL} — ACF of |Returns| (Volatility Clustering, 1m)', fontsize=14, fontweight='bold')

for j, src in enumerate(FUTURES_SOURCES):
    ax = axes[j]
    df = load_ohlcv(SYMBOL, '1m', src)
    if df.empty:
        continue
    r = df['returns'].dropna().abs().values
    nlags = min(120, len(r) // 4)
    acf_vals, confint = acf(r, nlags=nlags, alpha=0.05)
    
    ax.bar(range(nlags + 1), acf_vals, width=0.6,
           color=SOURCE_COLORS[src], alpha=0.7)
    ax.fill_between(range(nlags + 1),
                   confint[:, 0] - acf_vals,
                   confint[:, 1] - acf_vals,
                   alpha=0.15, color='blue')
    ax.axhline(0, color='black', linewidth=0.5)
    ax.set_title(f'{SOURCE_LABELS[src]} — 1m |returns| ACF')
    ax.set_xlabel('Lag (minutes)')
    ax.set_ylabel('ACF')

plt.tight_layout()
plt.show()

## 4. Intraday Seasonality

Volume and volatility patterns by hour of day (UTC). Identifies:
- When liquidity is highest (lower slippage)
- When volatility is highest (more opportunity)
- When to avoid trading

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle(f'{SYMBOL} — Intraday Seasonality (1m bars, by hour UTC)', fontsize=14, fontweight='bold')

bar_width = 0.13
offsets = {src: (i - 2.5) * bar_width for i, src in enumerate(SOURCES)}

for src in SOURCES:
    df = load_ohlcv(SYMBOL, '1m', src)
    if df.empty:
        continue
    df['hour'] = df['datetime'].dt.hour
    color = SOURCE_COLORS[src]
    label = SOURCE_LABELS[src]
    off = offsets[src]
    
    # Volume by hour
    hourly_vol = df.groupby('hour')['volume'].mean()
    axes[0].bar(hourly_vol.index + off, hourly_vol.values, width=bar_width,
               color=color, alpha=0.8, label=label)
    
    # Volatility by hour
    hourly_std = df.groupby('hour')['returns'].std()
    axes[1].bar(hourly_std.index + off, hourly_std.values * 100, width=bar_width,
               color=color, alpha=0.8, label=label)
    
    # Trade count by hour
    hourly_trades = df.groupby('hour')['trade_count'].mean()
    axes[2].bar(hourly_trades.index + off, hourly_trades.values, width=bar_width,
               color=color, alpha=0.8, label=label)

axes[0].set_title('Average Volume per 1m Bar')
axes[1].set_title('Return Volatility (std %)')
axes[2].set_title('Average Trade Count per 1m Bar')

for ax in axes:
    ax.set_xlabel('Hour (UTC)')
    ax.set_xticks(range(24))
    ax.legend(fontsize=7, ncol=2)

plt.tight_layout()
plt.show()

## 5. Cross-Exchange Analysis

Key questions across all 3 exchanges (Binance, Bybit, OKX):
- How correlated are prices across exchanges?
- Does one exchange lead the others? (lead-lag)
- What's the typical price spread between them?
- Futures vs spot basis per exchange

In [ ]:
# Load 1m OHLCV for all futures sources
futures_1m = {}
for src in FUTURES_SOURCES:
    futures_1m[src] = load_ohlcv(SYMBOL, '1m', src)
    print(f'  {SOURCE_LABELS[src]:20s}: {len(futures_1m[src]):,} bars')

# Merge all futures on timestamp
merged = futures_1m['binance_futures'][['timestamp_us', 'close', 'returns', 'volume']].rename(
    columns={'close': 'close_bn', 'returns': 'ret_bn', 'volume': 'vol_bn'})

for src, abbr in [('bybit_futures', 'bb'), ('okx_futures', 'okx')]:
    merged = merged.merge(
        futures_1m[src][['timestamp_us', 'close', 'returns', 'volume']].rename(
            columns={'close': f'close_{abbr}', 'returns': f'ret_{abbr}', 'volume': f'vol_{abbr}'}),
        on='timestamp_us', how='inner')

merged['datetime'] = pd.to_datetime(merged['timestamp_us'], unit='us', utc=True)
merged['spread_bn_bb'] = (merged['close_bn'] - merged['close_bb']) / merged['close_bn'] * 10000
merged['spread_bn_okx'] = (merged['close_bn'] - merged['close_okx']) / merged['close_bn'] * 10000
merged['spread_bb_okx'] = (merged['close_bb'] - merged['close_okx']) / merged['close_bb'] * 10000

print(f'\nMatched 1m bars across all 3 futures: {len(merged):,}')
print(f'\nPrice Correlations (close):')
for a, b in [('bn', 'bb'), ('bn', 'okx'), ('bb', 'okx')]:
    corr = merged[f'close_{a}'].corr(merged[f'close_{b}'])
    print(f'  {a.upper():>4s} ↔ {b.upper():<4s}: {corr:.10f}')

print(f'\nReturn Correlations (1m):')
for a, b in [('bn', 'bb'), ('bn', 'okx'), ('bb', 'okx')]:
    corr = merged[f'ret_{a}'].corr(merged[f'ret_{b}'])
    print(f'  {a.upper():>4s} ↔ {b.upper():<4s}: {corr:.6f}')

print(f'\nSpread Statistics (bps):')
for col, label in [('spread_bn_bb', 'Binance-Bybit'), ('spread_bn_okx', 'Binance-OKX'), ('spread_bb_okx', 'Bybit-OKX')]:
    s = merged[col]
    print(f'  {label:16s}: mean={s.mean():+.3f}  std={s.std():.3f}  '
          f'[{s.quantile(0.01):.2f}, {s.quantile(0.99):.2f}]')

# --- Plots ---
fig, axes = plt.subplots(2, 3, figsize=(18, 9))
fig.suptitle(f'{SYMBOL} — Cross-Exchange Futures (1m bars)', fontsize=14, fontweight='bold')

# Row 1: Spread distributions
for i, (col, label, color) in enumerate([
    ('spread_bn_bb', 'Binance − Bybit', '#F0B90B'),
    ('spread_bn_okx', 'Binance − OKX', '#00C4B4'),
    ('spread_bb_okx', 'Bybit − OKX', '#FF6B00'),
]):
    ax = axes[0, i]
    ax.hist(merged[col], bins=150, alpha=0.7, color=color, density=True)
    ax.axvline(0, color='red', linestyle='--', alpha=0.5)
    ax.set_title(f'{label} Spread (bps)')
    ax.set_xlabel('bps')

# Row 2: Return scatter plots
for i, (a, b, color) in enumerate([
    ('bn', 'bb', '#F0B90B'), ('bn', 'okx', '#00C4B4'), ('bb', 'okx', '#FF6B00')
]):
    ax = axes[1, i]
    ax.scatter(merged[f'ret_{a}'] * 100, merged[f'ret_{b}'] * 100,
              alpha=0.15, s=1, color=color)
    lim = 0.5
    ax.plot([-lim, lim], [-lim, lim], 'r--', alpha=0.5)
    ax.set_xlim(-lim, lim)
    ax.set_ylim(-lim, lim)
    ax.set_title(f'{a.upper()} vs {b.upper()} 1m Returns')
    ax.set_xlabel(f'{a.upper()} return (%)')
    ax.set_ylabel(f'{b.upper()} return (%)')

plt.tight_layout()
plt.show()

In [ ]:
# Lead-lag analysis across all 3 futures exchanges
print(f'{SYMBOL} — Lead-Lag Cross-Correlation (1m returns)')
print(f'{"─" * 70}')

pairs = [('bn', 'bb', 'Binance', 'Bybit'), ('bn', 'okx', 'Binance', 'OKX'), ('bb', 'okx', 'Bybit', 'OKX')]
lags = range(-10, 11)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle(f'{SYMBOL} — Lead-Lag: Futures 1m Returns', fontsize=14, fontweight='bold')

for idx, (a, b, a_name, b_name) in enumerate(pairs):
    ax = axes[idx]
    a_leads = []
    b_leads = []
    
    for lag in lags:
        if lag == 0:
            c = merged[f'ret_{a}'].corr(merged[f'ret_{b}'])
            a_leads.append(c)
            b_leads.append(c)
        else:
            c1 = merged[f'ret_{a}'].corr(merged[f'ret_{b}'].shift(-lag))
            c2 = merged[f'ret_{b}'].corr(merged[f'ret_{a}'].shift(-lag))
            a_leads.append(c1)
            b_leads.append(c2)
    
    ax.bar([l - 0.15 for l in lags], a_leads, width=0.3, alpha=0.7,
           color=SOURCE_COLORS[f'{a_name.lower()}_futures'],
           label=f'{a_name} leads')
    ax.bar([l + 0.15 for l in lags], b_leads, width=0.3, alpha=0.7,
           color=SOURCE_COLORS[f'{b_name.lower()}_futures'],
           label=f'{b_name} leads')
    ax.axhline(0, color='black', linewidth=0.5)
    ax.axvline(0, color='red', linestyle='--', alpha=0.3)
    ax.set_xlabel('Lag (minutes)')
    ax.set_ylabel('Cross-correlation')
    ax.set_title(f'{a_name} vs {b_name}')
    ax.legend(fontsize=8)
    
    # Print key lags
    print(f'\n  {a_name} vs {b_name}:')
    print(f'  {"Lag":>6s}  {f"{a_name} leads":>16s}  {f"{b_name} leads":>16s}')
    for lag_i, lag in enumerate(lags):
        if abs(lag) <= 3:
            marker = ' ← contemp.' if lag == 0 else ''
            print(f'  {lag:>6d}  {a_leads[lag_i]:>16.6f}  {b_leads[lag_i]:>16.6f}{marker}')

plt.tight_layout()
plt.show()

## 6. Volume Imbalance Profiling

Examine buy/sell volume ratio and its relationship to forward returns.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 9))
fig.suptitle(f'{SYMBOL} — Volume Imbalance Analysis (1m, Futures)', fontsize=14, fontweight='bold')

for j, src in enumerate(FUTURES_SOURCES):
    df = load_ohlcv(SYMBOL, '1m', src)
    if df.empty:
        continue
    
    df['volume_imbalance'] = (df['buy_volume'] - df['sell_volume']) / df['volume']
    df['fwd_return_1m'] = df['returns'].shift(-1)
    df['fwd_return_5m'] = df['close'].pct_change(5).shift(-5)
    
    clean = df.dropna(subset=['volume_imbalance', 'fwd_return_1m']).copy()
    clean['imb_bucket'] = pd.qcut(clean['volume_imbalance'], q=10, labels=False, duplicates='drop')
    
    color = SOURCE_COLORS[src]
    label = SOURCE_LABELS[src]
    
    # Imbalance distribution
    axes[0, j].hist(clean['volume_imbalance'], bins=100, alpha=0.7, color=color)
    axes[0, j].set_title(f'{label} — Imbalance Dist.')
    axes[0, j].set_xlabel('(Buy - Sell) / Total')
    
    # Forward return by imbalance bucket
    bucket_returns = clean.groupby('imb_bucket')['fwd_return_1m'].mean() * 10000
    axes[1, j].bar(bucket_returns.index, bucket_returns.values, color=color, alpha=0.7)
    axes[1, j].set_title(f'{label} — Fwd Return by Decile')
    axes[1, j].set_xlabel('Imbalance Decile (0=sell, 9=buy)')
    axes[1, j].set_ylabel('Fwd Return (bps)')
    axes[1, j].axhline(0, color='red', linestyle='--', alpha=0.5)
    
    # Correlations
    corr_1m = clean['volume_imbalance'].corr(clean['fwd_return_1m'])
    corr_5m = df.dropna(subset=['volume_imbalance', 'fwd_return_5m'])['volume_imbalance'].corr(
        df.dropna(subset=['volume_imbalance', 'fwd_return_5m'])['fwd_return_5m'])
    print(f'{label}: imbalance→fwd_1m corr={corr_1m:.6f}, imbalance→fwd_5m corr={corr_5m:.6f}')

plt.tight_layout()
plt.show()

## 7. Metrics Analysis: Funding & Open Interest

Examine Binance futures metrics for potential signals.

In [ ]:
metrics = load_metrics(SYMBOL)
ohlcv = load_ohlcv(SYMBOL, '5m', 'binance_futures')

if metrics.empty or ohlcv.empty:
    print('Missing metrics or OHLCV data')
else:
    # Merge metrics with 5m OHLCV
    merged_m = ohlcv[['timestamp_us', 'close', 'returns', 'volume']].merge(
        metrics[['timestamp_us', 'open_interest', 'open_interest_value',
                 'global_ls_ratio', 'top_trader_ls_ratio_positions']],
        on='timestamp_us', how='inner'
    )
    merged_m['datetime'] = pd.to_datetime(merged_m['timestamp_us'], unit='us', utc=True)
    merged_m['oi_change'] = merged_m['open_interest'].pct_change()
    merged_m['fwd_return'] = merged_m['returns'].shift(-1)
    
    print(f'{SYMBOL} — Binance Futures Metrics ({len(merged_m):,} matched 5m bars)')
    print(f'  OI range: {merged_m["open_interest"].min():,.0f} – {merged_m["open_interest"].max():,.0f}')
    print(f'  Global L/S ratio: {merged_m["global_ls_ratio"].min():.3f} – {merged_m["global_ls_ratio"].max():.3f}')
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(f'{SYMBOL} — Binance Futures Metrics', fontsize=14, fontweight='bold')
    
    # OI over time with price overlay
    ax1 = axes[0, 0]
    ax1.plot(merged_m['datetime'], merged_m['open_interest'], color='steelblue', alpha=0.8)
    ax1.set_title('Open Interest')
    ax1.set_ylabel('Contracts', color='steelblue')
    ax2 = ax1.twinx()
    ax2.plot(merged_m['datetime'], merged_m['close'], color='orange', alpha=0.5, linewidth=0.5)
    ax2.set_ylabel('Price', color='orange')
    
    # Global L/S ratio over time
    axes[0, 1].plot(merged_m['datetime'], merged_m['global_ls_ratio'], color='green', alpha=0.8, linewidth=0.5)
    axes[0, 1].axhline(1.0, color='red', linestyle='--', alpha=0.5)
    axes[0, 1].set_title('Global Long/Short Ratio')
    axes[0, 1].set_ylabel('L/S Ratio')
    
    # OI change vs forward return
    clean = merged_m.dropna(subset=['oi_change', 'fwd_return'])
    axes[1, 0].scatter(clean['oi_change'] * 100, clean['fwd_return'] * 10000,
                      alpha=0.3, s=3, color='steelblue')
    axes[1, 0].set_title('OI Change vs 5m Forward Return')
    axes[1, 0].set_xlabel('OI Change (%)')
    axes[1, 0].set_ylabel('Forward Return (bps)')
    axes[1, 0].set_xlim(-2, 2)
    axes[1, 0].set_ylim(-100, 100)
    axes[1, 0].axhline(0, color='red', linestyle='--', alpha=0.5)
    
    # L/S ratio vs forward return
    clean2 = merged_m.dropna(subset=['global_ls_ratio', 'fwd_return'])
    axes[1, 1].scatter(clean2['global_ls_ratio'], clean2['fwd_return'] * 10000,
                      alpha=0.3, s=3, color='green')
    axes[1, 1].set_title('Global L/S Ratio vs 5m Forward Return')
    axes[1, 1].set_xlabel('L/S Ratio')
    axes[1, 1].set_ylabel('Forward Return (bps)')
    axes[1, 1].axhline(0, color='red', linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    plt.show()
    
    # Correlations
    print(f'\nCorrelations with 5m forward return:')
    for col in ['oi_change', 'global_ls_ratio', 'top_trader_ls_ratio_positions']:
        if col in merged_m.columns:
            c = merged_m[col].corr(merged_m['fwd_return'])
            print(f'  {col:40s} {c:>10.6f}')

## Summary & Key Findings

After running this notebook on BTCUSDT (92 days, 6 sources), document:

1. **Return characteristics** — fat tails? skew? which timeframe has most structure?
2. **Autocorrelation** — any significant lags? momentum or mean-reversion?
3. **Volatility clustering** — how persistent? (informs position sizing)
4. **Seasonality** — best hours to trade?
5. **Cross-exchange** — which exchange leads? by how many minutes? spread distribution?
6. **Volume imbalance** — predictive of forward returns? which exchange has strongest signal?
7. **Metrics** — OI/L-S ratio signals worth pursuing?

→ Proceed to `02_lead_lag.ipynb` for deeper cross-exchange analysis  
→ Run same notebook for ETHUSDT and SOLUSDT once their parquet is built